### Target Variable : Tip with mutate variable -> tip_rate

In [ ]:
!cd

In [ ]:
!dir

In [ ]:
#!pip install PyMySQL==1.0.0

In [ ]:
#import pymysql.cursors
import pandas as pd
# Connect to the database

In [ ]:
# localhost or 127.0.0.1

In [ ]:
'''
# MySQL DB에서 데이터 받아와서 DataFrame에 저장

conn = pymysql.connect(host='localhost', user='root', 
                       password='8475', db='tip', charset='utf8',
                       autocommit=True, cursorclass=pymysql.cursors.DictCursor)
try:

    with conn.cursor() as curs:
      sql = "SELECT * FROM tips;"
      curs.execute(sql)
      rs = curs.fetchall()

      # DB에서 받아온 값을 DataFrame에 넣음

      df = pd.DataFrame(rs)
      print(df)
      #df.to_csv('query.csv')

finally:

   conn.close()
'''

In [ ]:
df = pd.read_csv('tips.csv')

In [ ]:
df.info()

In [ ]:
#df.isnull().sum()

### Encoding

In [ ]:
df['sex'].replace({'Female':0, 'Male':1}, inplace=True)
df["smoker"].replace({"No" : 0, "Yes" : 1}, inplace=True)
df["day"].replace({"Thur" : 0, "Fri" : 1, "Sat" : 2, "Sun" : 3}, inplace=True)
df["time"].replace({"Lunch" : 0, "Dinner" : 1}, inplace=True)

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
import numpy as np

In [ ]:
df.replace('',np.nan,inplace=True) # '' -> np.nan

In [ ]:
df.describe()

In [ ]:
type(df)

In [ ]:
df['tip_rate'] = ((df['tip']/df['total_bill'])*100).round(2)

In [ ]:
df['tip_rate'].head(3)

In [ ]:
df.describe()

In [ ]:
#import numpy as np
from sklearn.impute import SimpleImputer

In [ ]:
data = df.values

In [ ]:
y=df['tip'] # Target variable -> Tip

In [ ]:
X=df.drop('tip',axis=1)

In [ ]:
X.columns

In [ ]:
y.describe()

In [ ]:
data=X.values # Train data

In [ ]:
y=y.values

In [ ]:
import numpy as np

In [ ]:
sum(np.isnan(data).flatten())

In [ ]:
sum(np.isnan(y).flatten())

In [44]:
imputer = SimpleImputer(strategy='median')

In [45]:
imputer.fit(data)

SimpleImputer(strategy='median')

In [46]:
# transform the dataset
data_trans = imputer.transform(data) # Imputed data set

### 시작 

### RFE

In [47]:
from sklearn.feature_selection import RFE

In [ ]:
#from sklearn.tree import DecisionTreeClassifier

In [48]:
from sklearn.svm import SVR

In [49]:
estimator = SVR(kernel="linear")

In [50]:
# define RFE
rfe = RFE(estimator, n_features_to_select=4)

In [51]:
# fit RFE
selector=rfe.fit(data_trans, y)

In [ ]:
!cd

In [52]:
selector.support_

array([ True, False,  True, False, False,  True,  True])

In [53]:
# summarize all features
for i in range(data_trans.shape[1]):
  print('Column: %d,  Rank: %d' % (i,  rfe.ranking_[i]))

Column: 0,  Rank: 1
Column: 1,  Rank: 4
Column: 2,  Rank: 1
Column: 3,  Rank: 3
Column: 4,  Rank: 2
Column: 5,  Rank: 1
Column: 6,  Rank: 1


In [54]:
df1=pd.DataFrame(data_trans, columns=['total_bill', 'sex', 'smoker', 'day', 'time', 'size', 'tip_rate'])

In [55]:
df1.columns

Index(['total_bill', 'sex', 'smoker', 'day', 'time', 'size', 'tip_rate'], dtype='object')

In [56]:
df1.drop(['sex', 'day', 'time' ], axis=1)

,total_bill,smoker,size,tip_rate
0,16.99,0.0,2.0,5.94
1,10.34,0.0,3.0,16.05
2,21.01,0.0,3.0,16.66
3,23.68,0.0,2.0,13.98
4,24.59,0.0,4.0,14.68
...,...,...,...,...
239,29.03,0.0,3.0,20.39
240,27.18,1.0,2.0,7.36
241,22.67,1.0,2.0,8.82
242,17.82,0.0,2.0,9.82


### PCA

In [57]:
from sklearn.decomposition import PCA

In [71]:
df_pca = pd.read_csv('tips.csv')

In [74]:
df_pca['sex'].replace({'Female':0, 'Male':1}, inplace=True)
df_pca["smoker"].replace({"No" : 0, "Yes" : 1}, inplace=True)
df_pca["day"].replace({"Thur" : 0, "Fri" : 1, "Sat" : 2, "Sun" : 3}, inplace=True)
df_pca["time"].replace({"Lunch" : 0, "Dinner" : 1}, inplace=True)

In [75]:
df_pca['tip_rate'] = ((df['tip']/df['total_bill'])*100).round(2)

In [79]:
df_pca.describe()

,total_bill,tip,sex,smoker,day,time,size,tip_rate
count,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000
mean,19.785943,2.998279,0.643443,0.381148,1.725410,0.721311,2.569672,16.079754
std,8.902412,1.383638,0.479967,0.486667,1.155774,0.449276,0.951100,6.107020
min,3.070000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,3.560000
25%,13.347500,2.000000,0.000000,0.000000,0.000000,0.000000,2.000000,12.910000
50%,17.795000,2.900000,1.000000,0.000000,2.000000,1.000000,2.000000,15.475000
75%,24.127500,3.562500,1.000000,1.000000,3.000000,1.000000,3.000000,19.147500
max,50.810000,10.000000,1.000000,1.000000,3.000000,1.000000,6.000000,71.030000


In [77]:
data_pca = df_pca.values

In [78]:
data_pca.shape

(244, 8)

In [88]:
# define the transform
trans = PCA(n_components=5)

In [ ]:
# https://stackoverflow.com/questions/55582413/put-results-of-pca-into-dataframe

In [89]:
# transform the data
X_dim = trans.fit_transform(data_pca)

In [90]:
# summarize data after the transform
print(X_dim[:3, :])

[[ -0.68208966 -10.66377736  -1.45853912  -0.25823974  -0.10510125]
 [  8.86773204  -3.26463323  -1.6671699    0.98737732   0.14973529]
 [ -1.03924272   1.0668411   -1.29373308   0.40745874  -0.30338522]]


In [91]:
trans.components_

array([[-9.34200407e-01, -8.02043671e-02, -7.57463643e-03,
        -3.59379745e-03, -1.94744038e-02, -8.44758395e-03,
        -5.88598622e-02,  3.41835444e-01],
       [ 3.21436602e-01,  1.82320916e-01,  8.10781450e-04,
         6.49500983e-03,  2.36026233e-02,  5.60457143e-03,
         3.20612803e-02,  9.28320166e-01],
       [ 2.68972490e-02,  4.36920471e-02, -9.74144123e-02,
         2.29836474e-02, -9.36460327e-01, -3.26451049e-01,
        -6.10174953e-02,  9.91770546e-03],
       [-9.16108429e-02,  2.30182077e-01, -1.41377214e-02,
        -2.53547225e-01, -3.65301946e-02, -6.09416632e-02,
         9.31277241e-01, -4.25670544e-02],
       [ 1.01591458e-01, -8.76831690e-01, -7.74748850e-02,
         3.14548375e-01, -3.12111593e-02, -2.90361937e-02,
         3.13771126e-01,  1.25031067e-01]])

In [92]:
trans.explained_variance_ratio_.cumsum()

array([0.71740776, 0.97663321, 0.98863958, 0.99366119, 0.99630712])

In [93]:
trans.explained_variance_ratio_

array([0.71740776, 0.25922544, 0.01200637, 0.00502161, 0.00264593])

In [94]:
df_pca_transform = pd.DataFrame(X_dim)

In [95]:
df_pca_transform.describe().round(2)

,0,1,2,3,4
count,244.00,244.00,244.00,244.00,244.00
mean,0.00,0.00,0.00,0.00,0.00
std,9.33,5.61,1.21,0.78,0.57
min,-28.86,-10.66,-1.67,-2.29,-2.25
25%,-4.87,-3.48,-1.02,-0.41,-0.27
50%,2.12,-0.61,-0.38,-0.14,-0.07
75%,6.26,2.69,1.48,0.33,0.22
max,30.32,47.39,2.51,3.10,3.65


### Regression Feature Selection

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [ ]:
fs = SelectKBest(score_func=f_regression, k=4)

In [ ]:
data_trans.shape

In [ ]:
y.shape

In [ ]:
# apply feature selection
X_selected = fs.fit_transform(data_trans, y)

In [ ]:
parameters = fs.get_params(deep=True)

In [ ]:
parameters

In [ ]:
parameters['score_func']

In [ ]:
get_features = fs.fit(data_trans, y)

In [ ]:
get_features

In [ ]:
fs.get_support(indices=True)

In [ ]:
print(X_selected.shape)

In [ ]:
X_selected.shape[1]

In [ ]:
df_rfe_selected = pd.DataFrame(X_selected, columns=['total_bill', 'day', 'time', 'size'])

In [ ]:
df_rfe_selected.describe()